In [355]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
from scipy.sparse import csr_matrix
import warnings

In [31]:
df_symbol

,symbol_id,industry
0,3070,電子工業
1,3069,電子工業
2,3072,電子工業
3,3065,電子工業
4,3063,電子工業
...,...,...
16124,HEX.TO,None
16125,QVCC,NaN
16126,FINV,NaN
16127,020011,國內ETN


### merge industry to watchlist data

In [41]:
df_w = pd.read_excel('/Users/alanlin/Downloads/20200504/20191027-20200423_subscribe_wl.xlsx')

In [44]:
df_w = df_w.rename(columns={'user_id_y':'user_id'})

In [354]:
## get user watchlist symbols' industry
def test(x):
    industry = []
    for i in eval(x):
        #print(i)
        z = df_symbol[df_symbol['symbol_id']==i]['industry']
        ## change series to string
        z = z.to_string(index=False).replace(' ','')
        industry.append(z)
        
    return industry

In [89]:
df_w['industry_l'] = df_w['lists'].apply(test)

/Users/alanlin/.local/share/virtualenvs/py37-VhEGmDX6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
df_w['industry_count'] = df_w['industry_l'].apply(lambda x: Counter(x))

/Users/alanlin/.local/share/virtualenvs/py37-VhEGmDX6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
df_w2 = df_w[['user_id', 'industry_count', 'notification']]

In [100]:
df_w2

,user_id,industry_count,notification
0,6074,{'半導體業': 1},False
1,2198,"{'電腦及週邊設備業': 3, '其他': 2, '化學工業': 1, '電子零組件業': ...",False
2,2849,"{'通信網路業': 1, '半導體業': 1, '金融保險業': 1}",False
3,2198,{'': 7},False
4,6075,"{'其他': 2, '化學工業': 1, '電腦及週邊設備業': 1, '電子零組件業': ...",False
...,...,...,...
21515,576,"{'電腦及週邊設備業': 2, '電子零組件業': 2, '半導體業': 3, '資訊服務業...",False
21516,13369,"{'其他電子業': 1, '半導體業': 1}",False
21517,576,"{'生技醫療業': 8, '半導體業': 1, '油電燃氣業': 1, '光電業': 1}",False
21518,13370,{'半導體業': 1},False


### ignore warning message

In [105]:
warnings.filterwarnings('ignore')

### data preprocessing ( for features )

In [ ]:
df_w2['target'] = df_w2['notification'].apply(lambda x: 1 if x == True else 0)

In [119]:
df_w2['industry_count'] = df_w2['industry_count'].apply(lambda x: dict(x))

In [127]:
df_w2 = df_w2.reset_index(drop=True)

In [ ]:
data_all = []
for i in range(len(df_w2)):
    r = pd.DataFrame(df_w2['industry_count'][i], 
                     index=[df_w2['user_id'][i]]).T
    data_all.append(r)
    print(i)

In [ ]:
data_all2 = []
for i in range(len(df_w2)):
    r = pd.DataFrame(df_w2['industry_count'][i], 
                     index=[df_w2['user_id'][i]])
    data_all2.append(r)
    print(i)

In [149]:
rr = data_all[0]
i = 1

In [ ]:
while IndexError:
    rr = pd.concat([rr, data_all[i]], join='outer')
    print(i)
    i += 1

### merge all user industry data

In [177]:
df_all = pd.concat(data_all2)

### prepare X and y

In [181]:
df_feature = df_all.drop(columns=['', 'None', 'Series([],)', 'NaN'])

In [188]:
df_feature = df_feature.fillna(0)

In [213]:
df_feature

,半導體業,電腦及週邊設備業,其他,化學工業,電子零組件業,航運業,玻璃陶瓷,塑膠工業,電器電纜,貿易百貨,...,其他業,Energy,投信投顧公司,槓桿型ETF,國外成分證券ETF,Transportation,國內成分ETN,國外ETN,反向型ETF,管理股票
6074,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2198,3.0,3.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2849,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6075,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,3.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13369,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
576,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
df_target = df_w2[['user_id', 'target']]

In [217]:
df_target = df_target.set_index('user_id')

In [220]:
df_final = pd.concat([df_target, df_feature], axis=1)

In [221]:
X = df_final.iloc[:, 1:]
Y = df_final.iloc[:, :1]

In [224]:
X = np.array(X)
Y = np.array(Y)

In [230]:
Y.shape

(21477, 1)

In [232]:
X_csr = csr_matrix(X)

### import sklearn pakages

In [298]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

### train test split

In [330]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=87)

### Random Forest Classifier

In [343]:
ran = RandomForestClassifier()

In [344]:
ran.fit(X_train, y_train)

RandomForestClassifier()

In [345]:
y_predict = ran.predict(X_test)

In [346]:
accuracy_score(y_test, y_predict)

0.8947858472998138

* True Positive 真陽性 : 實際上是正樣本且被預測為正樣本
* True Negative 真陰性 : 實際上是負樣本且被預測為負樣本
* False Positive 假陽性 : 實際上是負樣本但被錯誤預測為正樣本
* False Negative 假陰性 : 實際上是正樣本但被錯誤預測為負樣本

In [347]:
print('Confusion Matrix:\n{}'.format(confusion_matrix(y_true=y_test, y_pred=y_predict)))
tp, fp, fn, tn = confusion_matrix(y_true=y_test, y_pred=y_predict).ravel()
print("True positives: " + str(tp))
print("False positives: " + str(fp))
print("True negatives: " + str(tn))
print("False negatives: " + str(fn))

Confusion Matrix:
[[3821   33]
 [ 419   23]]
True positives: 3821
False positives: 33
True negatives: 23
False negatives: 419


### SVC

In [348]:
clf = SVC(gamma='auto')

In [349]:
clf.fit(X_train, y_train)

SVC(gamma='auto')

In [350]:
y_predict = clf.predict(X_test)

In [351]:
accuracy_score(y_test, y_predict)

0.8982774674115456

In [352]:
print('Confusion Matrix:\n{}'.format(confusion_matrix(y_true=y_test, y_pred=y_predict)))
tp, fp, fn, tn = confusion_matrix(y_true=y_test, y_pred=y_predict).ravel()
print("True positives: " + str(tp))
print("False positives: " + str(fp))
print("True negatives: " + str(tn))
print("False negatives: " + str(fn))

Confusion Matrix:
[[3851    3]
 [ 434    8]]
True positives: 3851
False positives: 3
True negatives: 8
False negatives: 434


### Logistic Regression

In [279]:
log = LogisticRegression()

In [280]:
log.fit(X_train, y_train)

LogisticRegression()

In [281]:
y_predict = log.predict(X_test)

In [282]:
accuracy_score(y_predict, y_test)

0.8973463687150838

* GridSearchCV -> Tune Hyperparameters
* SMOTE -> unbalanced data problems